# Aprendizaje automático en GCP con BigQuery

![banner](https://github.com/langheran/GITMA2022/raw/main/images/banner.png)

## Objetivos

Este notebook ilustra los siguientes conceptos.

1. Crear un proyecto o usar una cuenta de servicio para conectarnos a _BigQuery_
2. Cargar en un `DataFrame` de `pandas` el resultado de un query en _BigQuery_ hacia una base de datos pública 
3. Usar _Aprendizaje Automático_ desde _BigQuery_
4. Hacer una predicción con un modelo creado con el lenguaje _BQML_
5. Incrementar la precisión de nuestro modelo con ingeniería de atributos

Cada objetivo corresponde a un #TODO en el notebook. Trata de completar todo el ejercicio antes de ver la solución.

Este laboratorio esta basado en esta lección de [Coursera](https://www.coursera.org/lecture/end-to-end-ml-tensorflow-gcp/lab-solution-bigquery-ml-klM24).

Este es el [jupyter notebook](https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive/06_structured/5_train_bqml.ipynb) original.

####  Costo de BigQuery

El free tier de BigQuery incluye procesamiento hasta 10 GB con `CREATE_MODEL`. Para más información ver las siguientes referencias:
- [BigQuery Sandbox](https://cloud.google.com/bigquery/docs/sandbox)
- [BigQuery Pricing](https://cloud.google.com/bigquery-ml/pricing)

## Requerimientos

### Paso 1:Instalar la librería Google Cloud Bigquery 

Primero instalamos la autenticación a GCP en `google-auth` y luego la _Python Client Library_ de BigQuery en `google-cloud-bigquery`.

In [ ]:
%%capture
!pip3 install google-auth
!pip3 install google-cloud-bigquery

### Paso 2: Instalar otros requerimientos

Ahora instalamos algunos requerimientos para correr el _jupyter_ y visualizar los datos como son, `ipykernel`, `tqdm`, `matplotlib`, etc...

In [ ]:
%%bash
cat requirements.txt

In [ ]:
%%capture
!pip3 install -r requirements.txt

### Paso 3. Incluir utilerías y cell magics

Ahora incluimos un par de utilerías para poder visualizar mejor la información.

In [ ]:
%run utils.py

También incluimos algunos `cell-magics`.

In [ ]:
%reload_ext fstring_magic
%reload_ext google.cloud.bigquery

## Conectar las Client Libraries de BigQuery

In [ ]:
credentials = None

### Paso 1: Crear proyecto en Google Cloud y habilitar BigQuery Sandbox

En caso de estar corriendo este lab en `cloudshell` efímero, o con una cuenta propia, en esta sección aprenderemos a crear un proyecto y habilitar la API de BigQuery.

In [ ]:

import sys
import os
os.environ['CLOUDSDK_PYTHON']=sys.executable

In [ ]:
USER = !(echo $$USER)
USER_HASH = abs(hash(USER[0])) % (10 ** 4)
PROJECT_ID = f"gitma2022-gcp-{USER_HASH}"
REGION = "us-central1"
BIGQUERY_SERVICE = "bigquery.googleapis.com"

In [ ]:
!gcloud config set disable_prompts true
if os.getenv("IS_TESTING"):
    !gcloud --quiet components install beta
    !gcloud --quiet components update

# TODO 1.: Rellena el código necesario para crear el proyecto

Con estas dos líneas configuramos `gcloud` para con el nuevo proyecto y región en _US_.

In [ ]:
!gcloud config set project $PROJECT_ID
!gcloud config set compute/region $REGION

Ahora, habilitamos la `BigQuery API`.

In [ ]:
 !gcloud services enable $BIGQUERY_SERVICE

In [ ]:
%%fstring
Listo! creamos un nuevo proyecto **{PROJECT_ID}**, ahora vamos a probar que tengamos conectividad con _BigQuery_.

In [ ]:
from google.cloud.bigquery import magics
magics.context.project = PROJECT_ID

In [ ]:
from google.cloud import bigquery
bq_client = bigquery.Client(
    project=PROJECT_ID,
)

In [ ]:
PROJECT_ID=!(gcloud config get-value core/project)
if isinstance(PROJECT_ID, list):
    PROJECT_ID = PROJECT_ID[0]
PROJECT_ID

Veamos un histograma de cuantos nacimientos hay en la base de datos `bigquery-public-data.samples.natality` por año.

In [ ]:
%%bigquery total_births
SELECT
    source_year AS year,
    COUNT(is_male) AS birth_count
FROM `bigquery-public-data.samples.natality`
GROUP BY year
ORDER BY year DESC
LIMIT 15

In [ ]:
%matplotlib inline

In [ ]:
total_births.plot(kind="bar", x="year", y="birth_count")

### Paso 2: Leer credenciales de un Service Account (Opcional)

En caso de estar corriendo este jupyter fuera del entorno de `cloudshell`, puedes optar por seguir los siguientes pasos

In [ ]:
key_filename="gitma-351819-e8d9daa363c5"
key_json_filename=f"{key_filename}.json"
key_encrypted_filename=f"{key_json_filename}.encrypted"

#### Crear una SA (service account) con Cloud SDK `gcloud`

In [ ]:
SERVICE_ACCOUNT_NAME="workshop"
ROLE="roles/bigquery.dataEditor"
ACCOUNT_MEMBER=f"serviceAccount:{SERVICE_ACCOUNT_NAME}@{PROJECT_ID}.iam.gserviceaccount.com"

Creamos la `service account`.

In [ ]:
!gcloud iam service-accounts create $SERVICE_ACCOUNT_NAME

Le asignamos el [rol](https://cloud.google.com/bigquery/docs/access-control) de `roles/bigquery.dataEditor` para que pueda usar la sentencia `CREATE_MODEL`. En caso de que quisieramos evitar la creación y entrenamiento de modelos, conviene usar el rol `roles/bigquery.user`.

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID --member=$ACCOUNT_MEMBER --role=$ROLE

Ahora descargamos la llave `json` con las credenciales de la SA.

In [ ]:
IAM_ACCOUNT = f"{SERVICE_ACCOUNT_NAME}@{PROJECT_ID}.iam.gserviceaccount.com"

In [ ]:
!gcloud iam service-accounts keys create $key_json_filename --iam-account=$IAM_ACCOUNT

In [ ]:
%%bash
cat $key_json_filename

#### ... o desencriptar la credencial

In [ ]:
def encrypt_key():
    import cryptocode
    with open(key_json_filename, 'r') as fp:
        content = fp.read()
    password = input('Enter your password...')
    enc_text = cryptocode.encrypt(content, password)
    with open(key_encrypted_filename, 'w') as f:
        f.write(enc_text)
# encrypt()

In [ ]:
import cryptocode

def decrypt_key():
    with open(key_encrypted_filename, 'r') as fp:
        enc_text = fp.read()

    password = input('Enter your password...')
    content = cryptocode.decrypt(enc_text, password)
    if not content:
        print('Password incorrecto...')
    else:
        with open(key_json_filename, 'w') as f:
            f.write(content)

decrypt_key()

#### Cargar una llave json a un objeto `service_account.Credentials`

In [ ]:
import os
from os.path import expanduser
home = expanduser("~")
load_key_credentials = not os.path.exists(f'{home}/cloudshell_open')
load_key_credentials

In [ ]:
import pandas as pd
from google.oauth2 import service_account
import json

if load_key_credentials:
    credentials = service_account.Credentials.from_service_account_file(
        key_json_filename,
    )
    PROJECT_ID = credentials.project_id
    PROJECT_ID

#### Inicializar objeto BigQuery client library con la SA

In [ ]:
if load_key_credentials:
    import os
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.abspath(key_json_filename)

In [ ]:
if credentials:
    from google.cloud import bigquery
    bq_client = bigquery.Client(
        credentials=credentials,
        project=PROJECT_ID,
    )

#### Cargar el cell magic de BigQuery con la SA (Opcional)

In [ ]:
%reload_ext google.cloud.bigquery
from google.cloud.bigquery import magics
magics.context.project = PROJECT_ID
if credentials:
    magics.context.credentials = credentials

## Explorar la base de datos `publicdata.samples.natality`

Hay 7 bases de datos públicas pre-cargadas en [BigQuery](https://console.cloud.google.com/bigquery?project=publicdata&ws=!1m4!1m3!3m2!1spublicdata!2ssamples) y se pueden encontrar en esta liga
https://cloud.google.com/bigquery/public-data.

Nosotros estamos interesados en la base de datos [natality](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=natality&page=table).

Hay varias formas de leer datos de _BigQuery_ a un `pandas.DataFrame`: con el [cell-magic](https://googleapis.dev/python/bigquery/latest/magics.html) `%%bigquery df` que vimos previamente, con el método de extensión `pandas.read_gbq` y con el `google.cloud.bigquery.Client`.

In [ ]:
query = """
SELECT
  year,
  COUNT(1) as num_babies
FROM
  publicdata.samples.natality
WHERE
  year > 2000
GROUP BY
  year
"""

Este es un ejemplo usando el [cell-magic](https://googleapis.dev/python/bigquery/latest/magics.html) de `%%bigquery df`

In [ ]:
%%bigquery df --params {"year": 2000}
SELECT
    *
FROM
  publicdata.samples.natality
WHERE
  year > @year
  AND gestation_weeks > 0
  AND mother_age > 0
  AND plurality > 0
  AND weight_pounds > 0
ORDER BY
  weight_pounds desc
LIMIT 10

El resultado es un `DataFrame` con los 10 bebes más pesados que nacieron a partir del año 2000.

In [ ]:
df.head()

Este es un ejemplo usando la extensión de `pandas.read_gbq`, aunque en este ejemplo no la usaremos.

In [ ]:
if credentials:
  df = pd.read_gbq(
      query,
      project_id=PROJECT_ID,
      credentials=credentials,
      dialect='standard',
  )
else:
  df = pd.read_gbq(
      query,
      project_id=PROJECT_ID,
      dialect='standard',
  )

El resultado es un `DataFrame` con los bebes que han nacido por año.

In [ ]:
df.head()

Este es un ejemplo usando el objeto `bq_client` que creamos previamente.

In [ ]:
df_valid = bq_client.query(query).to_dataframe()
# `head()` se usa para obtener las primeras filas de un DataFrame
display(df_valid.head())
# `describe()` se usa para generar la estadística descriptiva de un DataFrame
df_valid.describe()

Ahora [exploremos](https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive/06_structured/1_explore.ipynb) un poco los datos. Primero creamos una función que nos regrese solo la agrupación por cierto campo, su cuenta y su promedio (media).

In [ ]:
%%bigquery
SELECT
  AVG(weight_pounds) AS avg_wt
FROM
  publicdata.samples.natality
WHERE
  year > 2000

In [ ]:
def get_distinct_values(column_name):
  sql = """
SELECT
  {0},
  COUNT(1) AS num_babies,
  AVG(weight_pounds) AS avg_wt
FROM
  publicdata.samples.natality
WHERE
  year > 2000
GROUP BY
  {0}
  """.format(column_name)
  return bq_client.query(sql).to_dataframe()

#### Gráfica 1. Barras para visualizar `is_male` con respecto a su `avg_wt` y `num_babies`.

In [ ]:
# Bar plot to see is_male with avg_wt linear and num_babies logarithmic
df = get_distinct_values('is_male')
df.plot(x='is_male', y='num_babies', kind='bar');
df.plot(x='is_male', y='avg_wt', kind='bar');

#### Gráfica 2. Barras para visualizar `mother_age` con respecto a su `avg_wt` y `num_babies`.

In [ ]:
# Line plots to see mother_age with avg_wt linear and num_babies logarithmic
df = get_distinct_values('mother_age')
df = df.sort_values('mother_age')
df.plot(x='mother_age', y='num_babies');
df.plot(x='mother_age', y='avg_wt');

#### Gráfica 3. Barras para visualizar `plurality` (trillizos, mellizos, etc) con respecto a su `avg_wt` y `num_babies`.

In [ ]:
# Bar plot to see plurality(singleton, twins, etc.) with avg_wt linear and num_babies logarithmic
df = get_distinct_values('plurality')
df = df.sort_values('plurality')
df.plot(x='plurality', y='num_babies', logy=True, kind='bar');
df.plot(x='plurality', y='avg_wt', kind='bar');

#### Gráfica 4. Barras para visualizar `gestation_weeks` con respecto a su `avg_wt` y `num_babies`.

In [ ]:
# Bar plot to see gestation_weeks with avg_wt linear and num_babies logarithmic
df = get_distinct_values('gestation_weeks')
df = df.sort_values('gestation_weeks')
df.plot(x='gestation_weeks', y='num_babies', logy=True, kind='bar');
df.plot(x='gestation_weeks', y='avg_wt', kind='bar');

## Crear un modelo de aprendizaje automático en BigQuery

### Paso 1. Crear una base de datos de BigQuery en donde guardar el modelo

Dado que vamos a crear un modelo en `BigQuery` es necesario crear un `dataset` en el cual podamos guardarlo.

In [ ]:
from google.api_core import exceptions
dataset_id = "{}.demo".format(bq_client.project)
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"  # Multi-regional
try:
    # TODO 1.1. Rellena el código para crear la base de datos en Google Cloud  
    dataset = bq_client.create_dataset(, timeout=30)  # Make an API request.
    print("Created dataset {}.{}".format(bq_client.project, dataset.dataset_id))
except exceptions.Conflict as ex:
    ...

En caso de haber configurado la credencial como default en `gcloud` también es posible usar este comando para crear la base de datos.

```
bq --location=US mk -d demo
```

### Paso 2. Definir los features (atributos) a incluir en el modelo

Al ver las columnas de la base de datos, podemos observar que hay ciertas columnas que pueden ser de nuestro interés a la hora de predecir un aproximado en el peso del bebé.

Más aún, es posible fabricar nuevos features con la función `CAST` de BigQuery. En `BQML` todos los `strings` son considerados como features categóricos, y todos los numéricos como valores continuos. 

Agregamos la variable `hashmonth` para permitir el `split` (partición) de los datos de forma repetible y sin filtraciones (data leakage). Esto es porque queremos que todos los bebés que nacieron el mismo mes están ya sea en el `training` o en el `test` set y no regados entre ambos (de otra manera se estaría filtrando información, por ejemplo, en el caso de trillizos).

In [ ]:
%%bigquery
SELECT
    weight_pounds, -- this is the label; because it is continuous, we need to use regression
    CAST(is_male AS STRING) AS is_male,
    mother_age,
    CAST(plurality AS STRING) AS plurality,
    gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE
  year > 2000
  AND gestation_weeks > 0
  AND mother_age > 0
  AND plurality > 0
  AND weight_pounds > 0
LIMIT 10

### Paso 3. Crear un modelo con la sentencia BQML `CREATE MODEL`

Una vez que tenemos las columnas a considerar en nuestro modelo, es posible crear nuestro modelo usando `BQML` (BigQuery Machine Learning).

Usamos una [Common Table Expression](https://docs.microsoft.com/en-us/sql/t-sql/queries/with-common-table-expression-transact-sql?view=sql-server-ver16) o `CTE`, para facilitar el `SELECT` de los valores en el training set. 

Este proceso toma aproximadamente 4 minutos. 

In [ ]:
%%bigquery
CREATE or REPLACE MODEL demo.babyweight_model_asis
OPTIONS
  (
    -- TODO 3.1. Completa el tipo de modelo (model_type) a linear_reg
    model_type='',
    labels=['weight_pounds']
  ) AS
WITH natality_data AS (
  -- TODO 3.2 Completa la CTE con el query que diseñamos en la celda de arriba.
  
)

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    natality_data
WHERE
  ABS(MOD(hashmonth, 4)) < 3  -- select 75% of the data as training

75% de los datos son usados para entrenar, mientras el 25% lo reservamos para pruebas.

## Evaluar nuestro modelo

### Paso 1. Visualizar las estadísticas de nuestro modelo

In [ ]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(
    -- TODO 3.3. Rellena el nombre del modelo que acabamos de crear
    MODEL demo.
);

In [ ]:
cloud_console_link = f'https://console.cloud.google.com/bigquery?referrer=search&project={PROJECT_ID}&ws=!1m5!1m4!5m3!1s{PROJECT_ID}!2sdemo!3sbabyweight_model_asis'
display_link(cloud_console_link)

Algunas de estas columnas son [obvias](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-train#mltraining_info_output), como `loss` y `eval_loss`, que en este caso por ser regresión lineal, corresponden al `MSE` (mean square error) que corresponde a $\frac{1}{n} \Sigma_{i=1}^n({y}-\hat{y})^2$

Sin embargo, qué significan las demás?

**training_run** - Es cero si el modelo acaba de ser creado. Si el modelo es reentrenado usando la opción `warm_start`, este número incrementa con cada entrenamiento.

**iteration** - Relacionado con el `training_run`, comienza desde cero para cada iteración.

**duration_ms** - Indica cuantos milisegundos tomó la iteración.

### Paso 2. Predecir múltiples instancias desde la base de datos

In [ ]:
%%bigquery
SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    mother_age,
    CAST(plurality AS STRING) AS plurality,
    gestation_weeks
FROM
    publicdata.samples.natality
WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
LIMIT
    10

In [ ]:
%%bigquery
SELECT
  *
FROM
  ml.PREDICT(MODEL demo.babyweight_model_asis,
      (
      -- TODO 4.1. Rellena la clausula SELECT que acabamos de especificar en la celda de arriba 
      
    )
)
LIMIT 10

En la columna `predicted_weight_pounds` podemos ver el peso que nos está calculando el algoritmo.

### Paso 3. Predecir una sola instancia con datos personalizados

En caso que solo queramos predecir el valor de una sola instancia, es solo cuestión de llenar los datos en un cláusula `SELECT` con un alias para cada columna de entrada.

Evaluemos el caso en el que el bebé:
- Es hombre
- La madre tiene 28 años
- Es parto único
- Y tiene 38 semanas de gestación

In [ ]:
%%bigquery
SELECT
  *
FROM
  ml.PREDICT(MODEL demo.babyweight_model_asis,
      (SELECT
          'True' AS is_male,
          -- TODO 4.2. Especifica la edad de la madre (mother_age) en 28 años
          
          '1' AS plurality,
          38 AS gestation_weeks
    ))

### Paso 4. Evaluar nuestro modelo usando el 25% del dataset

In [ ]:
%%bigquery
SELECT
  *
FROM
  ML.EVALUATE(MODEL `demo.babyweight_model_asis`,
(
    WITH natality_data AS (
  SELECT
    weight_pounds,-- this is the label; because it is continuous, we need to use regression
    CAST(is_male AS STRING) AS is_male,
    mother_age,
    CAST(plurality AS STRING) AS plurality,
    gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
)

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    natality_data
WHERE
  -- TODO 4.3. Establece el test set para probar a 25% al igual el módulo a 3
  ABS(MOD(hashmonth, 4)) =   -- select 25% of the data as testing
)
)

Podemos ver que el modelo tiene una `MAE` (Mean Absolute Error, error absoluto medio, $\frac{1}{n} \Sigma_{i=1}^n|{y}-\hat{y}|$ ) de solo $0.82556$, considerando que el promedio del peso real es de $7.23$...

No está nada mal 😁!

### Paso 5. Explicar el resultado

Ahora usaremos la función [ML.EXPLAIN_PREDICT](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-explain-predict) para obtener las atribuciones de cada feature usando sus valores Shapley **en una única predicción**. Cómo se calculan estos valores queda fuera del scope de esta presentación, pero es posible consultarlos [aquí](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-xai-overview).

In [ ]:
%%bigquery explain_df
SELECT
  top_feature_attributions
FROM
  ML.EXPLAIN_PREDICT(
    MODEL `demo.babyweight_model_asis`,
    (
      SELECT
          'True' AS is_male,
          -- TODO 4.2. Especifica la edad de la madre (mother_age) en 28 años
          28 AS mother_age,
          '1' AS plurality,
          38 AS gestation_weeks
    ), STRUCT(10 AS top_k_features)
  )

In [ ]:
pd.DataFrame(explain_df.at[0, 'top_feature_attributions'].tolist())

Vemos que `plurality` fue el feature más determinante. Ahora veamos de forma global las atribuciones con [ML.GLOBAL_EXPLAIN](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-global-explain).

In [ ]:
%%bigquery
SELECT
  *
FROM
  ML.GLOBAL_EXPLAIN(MODEL `demo.babyweight_model_asis`)

Además de [exportar](https://cloud.google.com/bigquery-ml/docs/exporting-models) nuestro modelo a formato `SavedModel` de tensorflow, dado que es una regresión lineal también podemos examinar los [pesos](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-weights) directamente.

In [ ]:
%%bigquery
SELECT
  *
FROM
  ML.WEIGHTS(MODEL `demo.babyweight_model_asis`,
    STRUCT(true AS standardize))

$\hat{y} = -3609.900778 + 0.134131*mother\_age + gestation\_weeks*0.669343 + plurality\_weight(plurality) + is\_male\_weight(is\_male)$

## Agregar más features (atributos) al modelo

Dado que vimos que hay mucha variedad en la edad de las madres mayores a 45 años, conviene discretizarla.

In [ ]:
%%bigquery
SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    -- TODO 5.1. Incluye un varios IF para que menos de 18 sea LOW, y más de 45 high
    IF(mother_age < 18, ,
         IF(mother_age > 45, ,
            CAST(mother_age AS STRING))) AS mother_age,
    CAST(plurality AS STRING) AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
LIMIT 25

También conviene discretizar el campo `plurality` a valores ya sean `Multiple` o `Single`.

In [ ]:
%%bigquery
SELECT
    weight_pounds,
    'Unknown' AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    IF(plurality > 1, 'Multiple', 'Single') AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
LIMIT 25

También podemos tener el caso que el ultrasonido no se realizó y que no sabemos el sexo del feto. Por eso usamos dos `CTE` 's en la misma base de datos para simular esto.

In [ ]:
%%bigquery
WITH with_ultrasound AS (
  SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    CAST(plurality AS STRING) AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
),

without_ultrasound AS (
  SELECT
    weight_pounds,
    'Unknown' AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    IF(plurality > 1, 'Multiple', 'Single') AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
),

preprocessed AS (
  SELECT * from with_ultrasound
  UNION ALL
  SELECT * from without_ultrasound
)

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    preprocessed
WHERE
  ABS(MOD(hashmonth, 4)) < 3
LIMIT 25

Creemos este modelo con esas transformaciones. Tardará entre **5 y 10 mins**.

In [ ]:
%%bigquery
CREATE or REPLACE MODEL demo.babyweight_model_fc
OPTIONS
  (model_type='linear_reg', labels=['weight_pounds']) AS
WITH with_ultrasound AS (
  SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    CAST(plurality AS STRING) AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
),

without_ultrasound AS (
  SELECT
    weight_pounds,
    'Unknown' AS is_male,
    IF(mother_age < 18, 'LOW',
         IF(mother_age > 45, 'HIGH',
            CAST(mother_age AS STRING))) AS mother_age,
    IF(plurality > 1, 'Multiple', 'Single') AS plurality,
    CAST(gestation_weeks AS STRING) AS gestation_weeks,
    FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
),

preprocessed AS (
  SELECT * from with_ultrasound
  UNION ALL
  SELECT * from without_ultrasound
)

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    preprocessed
WHERE
  ABS(MOD(hashmonth, 4)) < 3

Es posible que te aparezca el siguiente mensaje y es porque se ha excedido el límite del free tier para `CREATE_MODEL`. En dado caso, tendrás que usar otra [cuenta](http://support.google.com/mail/answer/56256?hl=en) D: .

![limit](https://github.com/langheran/GITMA2022/raw/main/images/free_tier_limit.png)

In [ ]:
%%bigquery
SELECT * FROM ML.TRAINING_INFO(MODEL demo.babyweight_model_fc);

In [ ]:
cloud_console_link = f'https://console.cloud.google.com/bigquery?referrer=search&project={PROJECT_ID}&ws=!1m5!1m4!5m3!1s{PROJECT_ID}!2sdemo!3sbabyweight_model_fc'
display_link(cloud_console_link)

Evaluemos nuevamente el caso en el que el bebé:
- Es hombre
- La madre tiene 28 años
- Es parto único
- Y tiene 38 semanas de gestación

In [ ]:
%%bigquery
SELECT
  *
FROM
  ml.PREDICT(MODEL demo.babyweight_model_fc,
      (SELECT
          'True' AS is_male,
          -- TODO 5.2. Especifica la edad de la madre (mother_age) en 28 años

          '1' AS plurality,
          '38' AS gestation_weeks
    ))

## Cleanup

Ahora procederemos a borrar los modelos contenidos en el dataset `demo` de nuestro proyecto.

In [ ]:
dataset_id = "{}.demo".format(bq_client.project)
models = bq_client.list_models(dataset_id)  # Make an API request.

print("Models contained in '{}':".format(dataset_id))
for model in models:
    full_model_id = "{}.{}.{}".format(
        model.project, model.dataset_id, model.model_id
    )
    friendly_name = model.friendly_name
    print("Deleting {}".format(full_model_id))
    bq_client.delete_model(full_model_id)

## Conclusión

Felicidades, has concluido el taller! 🎉

En este taller aprendimos a:

1. Crear un proyecto o usar una cuenta de servicio para conectarnos a _BigQuery_
2. Cargar en un `DataFrame` de `pandas` el resultado de un query en _BigQuery_ hacia una base de datos pública 
3. Usar _Aprendizaje Automático_ desde _BigQuery_
4. Hacer una predicción con un modelo creado con el lenguaje _BQML_
5. Incrementar la precisión de nuestro modelo con ingeniería de atributos

Recuerda llenar el formulario en Google Forms para recibir tu certificado de participación.

Esperamos verte pronto con más habilidades de GCP 🤗.